# Self-Driving Cab

## Simulación de un Taxi Autónomo

Diseñemos una simulación de un taxi autónomo. El objetivo principal es demostrar, en un entorno simplificado, cómo se pueden utilizar técnicas de aprendizaje por refuerzo para desarrollar un enfoque eficiente y seguro para abordar este problema.

El trabajo del Smartcab es recoger al pasajero en una ubicación y dejarlo en otra. Aquí hay algunas cosas que nos encantaría que nuestro Smartcab se encargara:

- Dejar al pasajero en la ubicación correcta.
- Ahorrar tiempo al pasajero tomando el tiempo mínimo posible para dejarlo.
- Cuidar la seguridad del pasajero y las normas de tráfico.

Hay diferentes aspectos que deben considerarse aquí al modelar una solución de aprendizaje por refuerzo para este problema: recompensas, estados y acciones.


## Recompensa y Penalización

Dado que el agente (el conductor imaginario) está motivado por las recompensas y aprenderá a controlar el taxi mediante experiencias de prueba en el entorno, necesitamos decidir las recompensas y/o penalizaciones y su magnitud en consecuencia. Aquí hay algunos puntos a considerar:

* El agente debería recibir una recompensa positiva alta por una entrega exitosa porque este comportamiento es altamente deseado.
* El agente debería ser penalizado si intenta dejar al pasajero en ubicaciones incorrectas.
* El agente debería recibir una ligera penalización negativa por no llegar al destino después de cada paso de tiempo. "Ligera" negativa porque preferiríamos que nuestro agente llegue tarde en lugar de hacer movimientos incorrectos tratando de llegar al destino lo más rápido posible.


## State space

En Aprendizaje por Refuerzo, el agente encuentra un estado y luego toma una acción de acuerdo con el estado en el que se encuentra.

El Espacio de Estados es el conjunto de todas las situaciones posibles en las que nuestro taxi podría encontrarse. El estado debe contener información útil que el agente necesita para tomar la acción correcta.

Digamos que tenemos un área de entrenamiento para nuestro Smartcab donde lo estamos enseñando a transportar personas en un estacionamiento a cuatro ubicaciones diferentes (R, G, Y, B):


<img src="./img/Reinforcement_Learning_Taxi_Env.png" alt="drawing" width="600"/>

## Espacio de Estados

Supongamos que Smartcab es el único vehículo en este estacionamiento. Podemos dividir el estacionamiento en una cuadrícula de 5x5, lo que nos da 25 ubicaciones de taxi posibles. Estas 25 ubicaciones son una parte de nuestro espacio de estados. Observa que el estado de ubicación actual de nuestro taxi es la coordenada (3, 1).

También notarás que hay cuatro (4) ubicaciones donde podemos recoger y dejar a un pasajero: R, G, Y, B o [(0,0), (0,4), (4,0), (4,3)] en coordenadas (fila, columna). Nuestro pasajero ilustrado está en la ubicación Y y desea ir a la ubicación R.

Cuando también tenemos en cuenta un (1) estado adicional del pasajero de estar dentro del taxi, podemos tomar todas las combinaciones de ubicaciones de pasajeros y ubicaciones de destino para llegar a un número total de estados para nuestro entorno de taxi; hay cuatro (4) destinos y cinco (4 + 1) ubicaciones de pasajeros.


In [1]:
# So, our taxi environment has a total of possible states:
5*5*4*(4+1)

500

## Action space

## Acciones en el Entorno del Taxi

El agente encuentra uno de los 500 estados y toma una acción. La acción en nuestro caso puede ser moverse en una dirección o decidir recoger/dejar a un pasajero.

En otras palabras, tenemos seis acciones posibles:
1. sur
2. norte
3. este
4. oeste
5. recoger
6. dejar


## Espacio de Acciones

Este es el espacio de acciones: el conjunto de todas las acciones que nuestro agente puede tomar en un estado dado.

Observarás en la ilustración anterior que el taxi no puede realizar ciertas acciones en ciertos estados debido a las paredes. En el código del entorno, simplemente proporcionaremos una penalización de -1 por cada colisión con una pared y el taxi no se moverá en absoluto. Esto simplemente acumulará penalizaciones, haciendo que el taxi considere ir alrededor de la pared.


## Implementation

In [2]:
import gym
import time
import matplotlib.pyplot as plt

In [3]:
env = gym.make("Taxi-v3", render_mode="ansi").env
env.reset()
print(env.render())
# time.sleep(10)
# env.close()


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




## Interfaz del Entorno Gym

La interfaz central de Gym es `env`, que es la interfaz de entorno unificada. Los siguientes son los métodos de `env` que serían bastante útiles para nosotros:

* `env.reset`: Reinicia el entorno y devuelve un estado inicial aleatorio.
* `env.step(action)`: Avanza el entorno en un paso de tiempo. Devuelve
    - `observation`: Observaciones del entorno
    - `reward`: Si tu acción fue beneficiosa o no
    - `done`: Indica si hemos recogido y dejado con éxito a un pasajero, también llamado un episodio
    - `truncated`: Si el episodio se trunca debido a un límite de tiempo o una razón que no está definida.
    - `info`: Información adicional como rendimiento y latencia con fines de depuración
* `env.render`: Renderiza un fotograma del entorno (útil para visualizar el entorno)



## Reto

**Hay 4 ubicaciones (etiquetadas con diferentes letras), y nuestro trabajo es recoger al pasajero en una ubicación y dejarlo en otra. Recibimos +20 puntos por una entrega exitosa y perdemos 1 punto por cada paso de tiempo que toma. También hay una penalización de 10 puntos por acciones de recogida y entrega ilegales.**


In [4]:
env.reset() # reset environment to a new, random state
print(env.render())

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Action Space Discrete(6)
State Space Discrete(500)


* El cuadrado lleno representa el taxi, que es amarillo sin pasajero y verde con pasajero.
* El símbolo "|" representa una pared que el taxi no puede cruzar.
* R, G, Y, B son las posibles ubicaciones de recogida y destino. La letra azul representa la ubicación actual de recogida del pasajero, y la letra morada es el destino actual.


Según lo verificado por las impresiones, tenemos un Espacio de Acciones de tamaño 6 y un Espacio de Estados de tamaño 500. Como verás, nuestro algoritmo de aprendizaje por refuerzo no necesitará más información que estas dos cosas. Todo lo que necesitamos es una forma de identificar un estado de manera única asignando un número único a cada estado posible, y el aprendizaje por refuerzo aprende a elegir un número de acción de 0 a 5 donde:

0 = sur
1 = norte
2 = este
3 = oeste
4 = recoger
5 = dejar


Recuerda que los 500 estados corresponden a una codificación de la ubicación del taxi, la ubicación del pasajero y la ubicación de destino.

El Aprendizaje por Refuerzo aprenderá un mapeo de estados a la acción óptima a realizar en ese estado mediante la exploración, es decir, el agente explora el entorno y toma acciones basadas en las recompensas definidas en el entorno.

La acción óptima para cada estado es la acción que tiene la recompensa acumulativa a largo plazo más alta.


De hecho, podemos tomar nuestra ilustración anterior, codificar su estado y dárselo al entorno para que lo renderice en Gym. Recuerda que tenemos el taxi en la fila 3, columna 1, nuestro pasajero está en la ubicación 2 y nuestro destino está en la ubicación 0. Usando el método de codificación de estado Taxi-v3, podemos hacer lo siguiente:


In [5]:
state = env.encode(3, 1, 1, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.unwrapped.s = state
print(env.render())

State: 324
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [6]:
# Crea un state donde el taxi se encuentra en la fila 1, columna 3, y el pasajero está en la G con Destino Y
state = env.encode(1, 3, 1, 2) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.unwrapped.s = state
print(env.render())


State: 166
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




Estamos utilizando las coordenadas de nuestra ilustración para generar un número correspondiente a un estado entre 0 y 499, lo cual resulta ser 324 para el estado de nuestra ilustración.

Luego podemos establecer manualmente el estado del entorno utilizando env.unwrapped.s con ese número codificado. Puedes experimentar con los números y verás que el taxi, el pasajero y el destino se mueven.


## The Reward Table

Cuando se crea el entorno Taxi, también se crea una tabla de recompensas inicial llamada `P`. Podemos pensar en ella como una matriz que tiene el número de estados como filas y el número de acciones como columnas.

Dado que cada estado está en esta matriz, podemos ver los valores de recompensa predeterminados asignados al estado de nuestra ilustración:


In [7]:
state = env.encode(3, 1, 1, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.unwrapped.s = state
print(env.render())

State: 324
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [8]:
env.P[324]

{0: [(1.0, 424, -1, False)],
 1: [(1.0, 224, -1, False)],
 2: [(1.0, 344, -1, False)],
 3: [(1.0, 324, -1, False)],
 4: [(1.0, 324, -10, False)],
 5: [(1.0, 324, -10, False)]}

Este diccionario tiene la estructura {acción: [(probabilidad, próximo estado, recompensa, hecho)]}.

Algunas cosas a tener en cuenta:

* Los números del 0 al 5 corresponden a las acciones (sur, norte, este, oeste, recoger, dejar) que el taxi puede realizar en nuestro estado actual en la ilustración.
* En este entorno, la probabilidad siempre es 1.0.
* El próximo estado es el estado en el que estaríamos si tomamos la acción en este índice del diccionario.
* Todas las acciones de movimiento tienen una recompensa de -1 y las acciones de recoger/dejar tienen una recompensa de -10 en este estado en particular. Si estamos en un estado donde el taxi tiene un pasajero y está encima del destino correcto, veríamos una recompensa de 20 en la acción de dejar (5).
* done se utiliza para indicarnos cuándo hemos dejado con éxito a un pasajero en la ubicación correcta. Cada entrega exitosa es el final de un episodio.

Ten en cuenta que si nuestro agente elige explorar la acción dos (2) en este estado, estaría yendo hacia el Este, hacia una pared. El código fuente ha hecho imposible mover realmente el taxi a través de una pared, así que si el taxi elige esa acción, simplemente seguirá acumulando penalizaciones de -1, lo que afecta a la recompensa a largo plazo.


## Without Reinforcement Learning

In [9]:
env = gym.make("Taxi-v3", render_mode="ansi")
env.reset()
env.unwrapped.s = 324  # set environment to illustration's state


epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, trunc, info = env.step(action)
    # Añade +1 a la variable penaltie cuando el reward sea -10
    if reward == -10:
        penalties = penalties + 1
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 2345
Penalties incurred: 774


In [10]:
frames

[{'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| :\x1b \x1b: : : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (North)\n',
  'state': 224,
  'action': 1,
  'reward': -1},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| :\x1b \x1b: : : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (Dropoff)\n',
  'state': 224,
  'action': 5,
  'reward': -10},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| |\x1b \x1b: | : |\n|Y| : |B: |\n+---------+\n  (South)\n',
  'state': 324,
  'action': 0,
  'reward': -1},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| |\x1b \x1b: | : |\n|Y| : |B: |\n+---------+\n  (Pickup)\n',
  'state': 324,
  'action': 4,
  'reward': -10},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| |\x1b \x1b: | : |\n|Y| : |B: |\n+---------+\n  (Dropoff)\n',
  'state': 324,
  'action': 5,
  'reward': -10},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG

In [11]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        time.sleep(.2)

In [12]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.2)
        
print_frames(frames)

## With Reinforcement Learning

Vamos a utilizar un algoritmo simple de aprendizaje por refuerzo llamado Q-learning, que le dará a nuestro agente algo de memoria.


Básicamente, el Q-learning permite que el agente utilice las recompensas del entorno para aprender, con el tiempo, la mejor acción a tomar en un estado dado.

En nuestro entorno Taxi, tenemos la tabla de recompensas, P, de la que el agente aprenderá. Lo hace al recibir una recompensa por tomar una acción en el estado actual, y luego actualiza un valor Q para recordar si esa acción fue beneficiosa.

Los valores almacenados en la tabla Q se llaman valores Q, y se corresponden con una combinación (estado, acción).

Un valor Q para una combinación particular de estado-acción es representativo de la "calidad" de una acción tomada desde ese estado. Mejores valores Q implican mejores posibilidades de obtener recompensas mayores.

Por ejemplo, si el taxi se enfrenta a un estado que incluye un pasajero en su ubicación actual, es muy probable que el valor Q para recoger sea más alto en comparación con otras acciones, como dejar o ir hacia el norte.


Estamos asignando (), o actualizando, el valor Q del estado y la acción actual del agente primero tomando un peso () del antiguo valor Q, y luego añadiendo el valor aprendido. El valor aprendido es una combinación de la recompensa por tomar la acción actual en el estado actual, y la recompensa máxima descontada del próximo estado en el que estaremos una vez que tomemos la acción actual.

Básicamente, estamos aprendiendo la acción adecuada a tomar en el estado actual al observar la recompensa para la combinación estado/acción actual y las máximas recompensas para el próximo estado. Esto eventualmente hará que nuestro taxi considere la ruta con las mejores recompensas concatenadas.

El valor Q de un par estado-acción es la suma de la recompensa instantánea y la recompensa futura descontada (del estado resultante). La forma en que almacenamos los valores Q para cada estado y acción es a través de una tabla Q.


### Q-table

La tabla Q es una matriz donde tenemos una fila para cada estado (500) y una columna para cada acción (6). Se inicializa primero en 0, y luego los valores se actualizan después del entrenamiento. Ten en cuenta que la tabla Q tiene las mismas dimensiones que la tabla de recompensas, pero tiene un propósito completamente diferente.


<img src="./img/q-matrix-initialized-to-learned_gQq0BFs.png" alt="drawing" width="650"/>

Desglosándolo en pasos, obtenemos:

* Inicializar la tabla Q con todos los valores en cero.
* Comenzar a explorar acciones: Para cada estado, seleccionar una de entre todas las acciones posibles para el estado actual (S).
* Viajar al siguiente estado (S') como resultado de esa acción (a).
* Para todas las acciones posibles desde el estado (S'), seleccionar aquella con el valor Q más alto.
* Actualizar los valores de la tabla Q usando la ecuación.
* Establecer el siguiente estado como el estado actual.
* Si se alcanza el estado objetivo, entonces finalizar y repetir el proceso.


Después de suficiente exploración aleatoria de acciones, los valores Q tienden a converger, sirviendo a nuestro agente como una función de valor de acción que puede explotar para elegir la acción más óptima a partir de un estado dado.

Existe un compromiso entre la exploración (elegir una acción al azar) y la explotación (elegir acciones basadas en los valores Q ya aprendidos). Queremos evitar que la acción siempre tome la misma ruta, y posiblemente sobreajuste, así que introduciremos otro parámetro llamado "epsilon" para atender a esto durante el entrenamiento.

En lugar de simplemente seleccionar la mejor acción aprendida con valor Q, a veces favoreceremos la exploración adicional del espacio de acciones. Un valor epsilon más bajo resulta en episodios con más penalizaciones (en promedio), lo que es obvio porque estamos explorando y tomando decisiones al azar.


## Vamos a construirlo!

In [13]:
import numpy as np
# Crea un array2d de ceros del tamaño de los diferentes estados y las diferentes posibles acciones
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [14]:
q_table.shape

(500, 6)

In [15]:
q_table.size

3000

Ahora podemos crear el algoritmo de entrenamiento que actualizará esta tabla Q a medida que el agente explore el entorno a lo largo de miles de episodios.

En la primera parte de while not done, decidimos si elegir una acción al azar o explotar los valores Q ya calculados. Esto se hace simplemente utilizando el valor de epsilon y comparándolo con la función random.uniform(0, 1), que devuelve un número arbitrario entre 0 y 1.

Ejecutamos la acción elegida en el entorno para obtener el próximo estado y la recompensa por realizar la acción. Después de eso, calculamos el valor Q máximo para las acciones correspondientes al próximo estado, y con eso, podemos actualizar fácilmente nuestro valor Q al nuevo valor_q:


In [16]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [17]:
# %%time
# """Training the agent"""

import random
from IPython.display import clear_output
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 10001):
    state = env.reset()[0]

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, trunc, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 10000
Training finished.



Ahora que la tabla Q se ha establecido durante 100,000 episodios, veamos cuáles son los valores Q en el estado de nuestra ilustración:

In [18]:
env.P[324]

{0: [(1.0, 424, -1, False)],
 1: [(1.0, 224, -1, False)],
 2: [(1.0, 344, -1, False)],
 3: [(1.0, 324, -1, False)],
 4: [(1.0, 324, -10, False)],
 5: [(1.0, 324, -10, False)]}

In [19]:
# Accede a los valores de la q-table del estado 324
q_table[324]

array([-2.48911514, -2.48864036, -2.48867708, -2.48930743, -8.41229561,
       -8.73904794])

In [20]:
np.argmax(q_table[324])

1

El valor máximo de Q es "north" o "east" (-2.489), ¡así que parece que Q-learning ha aprendido efectivamente la mejor acción a tomar en el estado de nuestra ilustración!


### Evaluate

Vamos a evaluar el rendimiento de nuestro agente. Ya no necesitamos explorar acciones más, así que ahora la siguiente acción siempre se selecciona usando el mejor valor Q:


In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 10
frames = []
for _ in range(episodes):
    env.reset()
    # Crea el estado inicial
    # state = env.encode(3, 1, 2, 0)
    # env.reset()
    # env.s = state
    # Inicializa las epochs, penalties y rewards
    epochs, penalties, reward = 0, 0, 0

    done = False
    actions = []
    while not done:
        
        # Elige la accion que te indique el maximo valor de la q_table
        action = np.argmax(q_table[state])
        actions.append(action)
        # Ejecuta la accion
        state, reward, done, trunc, info = env.step(action)
        
        frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
        )
        # Actualiza el valor de penalties si el reward es -10
        if reward == -10:
            penalties += 1

        epochs += 1
        
    # frames = []
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

In [212]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 129
State: 410
Action: 5
Reward: 20


In [213]:
actions

[0, 1, 1, 3, 1, 4, 0, 0, 0, 0, 5]

Podemos ver en la evaluación que el rendimiento del agente mejoró significativamente y no incurrió en penalizaciones, lo que significa que realizó las acciones correctas de recogida/dejar con 100 pasajeros diferentes.


In [214]:
env.action_space.sample()

3

In [19]:
"""Evaluate agent's performance without Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    env.reset()
    # Crea el estado inicial
    state = env.encode(3, 1, 2, 0)
    env.s = state
    # Inicializa las epochs, penalties y rewards
    epochs, penalties, reward = 0, 0, 0

    done = False
    actions = []
    while not done:
        # Elige la acción random
        action = env.action_space.sample()
        actions.append(action)
        # Ejecuta la accion
        state, reward, done, trunc, info = env.step(action)

        # Actualiza el valor de penalties si el reward es -10
        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 2737.22
Average penalties per episode: 884.9


# SARSA (State-Action-Reward-State-Action)

SARSA es un algoritmo específico de aprendizaje por refuerzo que se utiliza para actualizar los valores de acción en función de la observación del siguiente estado y acción, además de la recompensa actual. En SARSA, se elige una acción (A) en un estado (S), se observa el siguiente estado (S') y se elige una nueva acción (A') basada en una política de toma de decisiones (que puede ser ε-greedy, por ejemplo). Luego, se actualizan los valores de acción utilizando la recompensa recibida y el valor de acción del siguiente estado y acción.



La principal diferencia entre Q-Table y SARSA radica en cómo se actualizan los valores de acción.
En la Q-Table, los valores se actualizan considerando el máximo valor de acción posible en el siguiente estado, independientemente de la acción tomada. En cambio, SARSA actualiza los valores de acción utilizando la acción real tomada en el siguiente estado.
Por lo tanto, mientras que Q-Table es un método off-policy (actualiza los valores de acción considerando la mejor acción posible), SARSA es un método on-policy (actualiza los valores de acción considerando la acción real tomada).


Q-Table:
- Ventajas: Es simple de entender e implementar en entornos con un número limitado de estados y acciones.
- Inconvenientes: No es escalable para entornos con un gran número de estados y acciones debido a la necesidad de almacenar y actualizar una tabla grande de valores de acción.

SARSA:
- Ventajas: Es más eficiente en términos de memoria y puede escalar mejor a entornos con un gran número de estados y acciones.
- Inconvenientes: Puede ser más difícil de implementar y entender en comparación con Q-Table debido a la necesidad de seguir una política de toma de decisiones y actualizar los valores de acción de manera adecuada.

In [20]:
# Inicialización de la tabla Q
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hiperparámetros
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# Para el registro de métricas
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()[0]
    action = env.action_space.sample() if random.uniform(0, 1) < epsilon else np.argmax(q_table[state])

    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        next_state, reward, done, trunc, info = env.step(action) 
        next_action = env.action_space.sample() if random.uniform(0, 1) < epsilon else np.argmax(q_table[next_state])

        old_value = q_table[state, action]
        next_value = q_table[next_state, next_action]
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_value)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        action = next_action
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.



In [ ]:
# # Q-Learning
# action = np.argmax(q_table[state]) # Exploit learned values

# # SARSA
# next_action = np.argmax(q_table[next_state]) # Choose next action

## Alpha (α):
El parámetro alpha controla la tasa de aprendizaje en los algoritmos de aprendizaje por refuerzo. Es una medida de cuánto confiamos en las nuevas actualizaciones de los valores de acción en comparación con los valores existentes.

- Si alpha es alto, damos más peso a las nuevas recompensas para actualizar los valores de acción.
- Si alpha es bajo, damos más peso a los valores de acción existentes y aprendemos más lentamente.

Ejemplo: Imagina que estás aprendiendo a jugar al ajedrez. Si tienes un alpha alto, estarías cambiando tus estrategias rápidamente después de cada partida. Si tienes un alpha bajo, estarías más inclinado a mantener tus estrategias existentes durante más tiempo, incluso si no te están dando buenos resultados.

## Gamma (γ):
El parámetro gamma es el factor de descuento en los algoritmos de aprendizaje por refuerzo. Controla cuánto valoramos las recompensas futuras en comparación con las recompensas inmediatas.
- Un gamma cercano a 1 significa que valoramos mucho las recompensas futuras.
- Un gamma cercano a 0 significa que solo valoramos las recompensas inmediatas.

Ejemplo: Imagina que estás decidiendo si estudiar para un examen o salir con tus amigos. Si tienes un gamma alto, estarías más inclinado a estudiar porque valoras mucho las recompensas futuras (buenas calificaciones). Si tienes un gamma bajo, estarías más inclinado a salir con tus amigos porque solo valoras la recompensa inmediata (diversión).

## Epsilon (ε):
El parámetro epsilon controla la exploración frente a la explotación en los algoritmos de aprendizaje por refuerzo. Determina la probabilidad de elegir una acción al azar en lugar de la acción óptima según los valores de acción actuales.
- Un epsilon alto significa que somos más propensos a explorar nuevas acciones.
- Un epsilon bajo significa que somos más propensos a explotar las acciones conocidas.

Ejemplo: Imagina que estás decidiendo qué película ver en Netflix. Si tienes un epsilon alto, es más probable que explores nuevas películas en lugar de ver tus favoritas. Si tienes un epsilon bajo, es más probable que veas tus películas favoritas una y otra vez sin explorar nuevas opciones.